# CMIP6 heat load for cows at the country level with additional cows on new land

In [ ]:
!pip install xarray==0.17.0

In [ ]:
%%capture
!pip install netcdf4
import os, fnmatch, numpy as np, pandas as pd, xarray as xr
import pylab as plt, plotly.express as px, seaborn as sns
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
import cv2; from google.colab.patches import cv2_imshow
%load_ext google.colab.data_table 
from scipy.interpolate import make_interp_spline

#!apt-get -qq install libgeos-dev
#!pip install -qq https://github.com/matplotlib/basemap/archive/master.zip
#from mpl_toolkits.basemap import Basemap, maskoceans, shiftgrid

!pip install zarr gcsfs cftime nc-time-axis 
import gcsfs, zarr; gcs = gcsfs.GCSFileSystem( token = 'anon' )

from google.colab import drive; drive.mount( '/content/drive' )

In [ ]:
def rgb2hex( rgbstring ): 
    s = rgbstring.split( '(' )[1].split( ',' )
    return "#{0:02x}{1:02x}{2:02x}".format(max(0,min(np.int(s[0]),255)),max(0,min(np.int(s[1]),255)),max(0,min(np.int(s[2].split(')')[0]),255)))

def canonical2continent(country):
    if country in ["Algeria", "Angola", "Benin", "Botswana", "Burkina", "Burundi", "Cameroon", "Cape Verde", "Central African Republic", 
                   "Chad", "Comoros", "Congo", "Congo, Democratic Republic of", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", 
                   "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", 
                   "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", 
                   "Nigeria", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", 
                   "South Sudan", "Sudan", "Swaziland", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe", "Canary Isl. (Spain)", 
                   "Congo, R. of", "Cote d'Ivoire", "France (Mayotte)", "France (Réunion)", "France (Mozambique Channel Isl.)"]:
        return "Africa"

    elif country in ["Afghanistan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Burma (Myanmar)", "Cambodia", "China", "East Timor", "India", 
                     "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Korea, North", "Korea, South", "Kuwait", 
                     "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", "Nepal", "Oman", "Pakistan", "Philippines", "Qatar", 
                     "Russian Federation", "Saudi Arabia", "Singapore", "Sri Lanka", "Syria", "Tajikistan", "Thailand", "Turkey", "Turkmenistan", 
                     "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen", "Hong Kong", "Malaysia (Peninsula East)", "Korea (South)", 
                     "Turkey (Black Sea)", "Saudi Arabia (Persian Gulf)", "Myanmar", "Taiwan", "Russia", "Andaman & Nicobar Isl. (India)", "Korea (North)"]:
        return "Asia"

    elif country in ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", 
                     "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland", 
                     "Ireland", "Italy", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", 
                     "Netherlands", "Norway", "Poland", "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", 
                     "Switzerland", "Ukraine", "United Kingdom", "Vatican City", "Norway (Svalbard Isl.)", "Channel Isl. (UK)"]:
        return "Europe"

    elif country in ["Antigua and Barbuda", "Bahamas", "Barbados", "Belize", "Canada", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "El Salvador", 
                     "Grenada", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Saint Kitts and Nevis", "Saint Lucia", 
                     "Saint Vincent and the Grenadines", "Trinidad and Tobago", "United States", "Alaska (USA)", "Greenland", "Haiti (Navassa Isl.)", 
                     "Puerto Rico (USA)", "France (Martinique)", "Turks & Caicos Isl. (UK)", "US Virgin Isl.", "France (Guadeloupe)", "Antigua & Barbuda", 
                     "British Virgin Isl. (UK)", "Anguilla (UK)", "Saint Vincent & the Grenadines", "Bermuda (UK)", "Cayman Isl. (UK)", "Montserrat (UK)"]:
        return "North America"

    elif country in ["Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "New Zealand", "Palau", "Papua New Guinea", "Samoa", 
                     "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu", "American Samoa", "Australia (Heard & McDonald Isl.)", "Hawaii", 
                     "Australia (Macquarie Isl.)", "Kermadec Isl. (New Zealand)", "Australia (Lord Howe Isl.)", "Marshall Isl.", "France (New Caledonia)", 
                     "Solomon Isl.", "Palmyra Atoll & Kingman Reef (USA)", "Brunei Darussalam", "Guam (USA)", "Howland & Baker Isl. (USA)", 
                     "France (French Polynesia)", "Australia (Christmas Isl.)", "Johnston Atoll (USA)", "Northern Marianas (USA)", "Cook Isl. (New Zealand)", 
                     "Timor Leste", "Australia (Norfolk Isl.)", "France (Wallis & Futuna Isl.)"]:
        return "Oceania"

    elif country in ["Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela", 
                     "Falkland Isl. (UK)", "France (French Guiana)", "Netherlands Antilles (Windward)", "Trinidad & Tobago", "Chile (Desventuradas Isl.)", 
                     "Brazil (Trindade & Martin Vaz Isl.)"]:
        return "South America"

    elif country in ["Chagos Archipel., Brit. Ind. Oc. Terr. (UK)", "Azores Isl. (Portugal)", "Chile (J. Fernandez, Felix and Ambrosio Isl.)", 
                     "Ecuador (Galapagos Isl.)", "Amsterdam & St Paul Isl. (France)", "Madeira Isl. (Portugal)", "Jarvis Isl. (USA)", 
                     "Australia (Cocos (Keeling) Isl.)", "Tristan da Cunha Isl. (UK)", "Chile (Easter Isl.)", "South Georgia & Sandwich Isl. (UK)", 
                     "Denmark (Faeroe Isl.)", "Jan Mayen Isl. (Norway)", "France (Kerguelen Isl.)"]:
        return "Open Ocean"
    else: return None

### Array of grid areas ###
nlon = 1440
lat = np.linspace( -89.875, 89.875, 720) * np.pi / 180 # in radians
area = ( 2 * np.pi * 6.3781e3 ) / ( ( lat.shape[0] + 1 ) * 2 ) * ( 2 * np.pi *  6.3781e3 * np.cos(lat) ) / ( nlon + 1 )
cellarea = np.repeat( area[:,np.newaxis], nlon, axis = 1 )

# 1. SSP livestock number increase at the regional level

In [ ]:
### SSP projections from IIASA ###
iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
iam.year = pd.to_numeric( iam.year )

### get percent change from 2010 baseline ###
iam2010 = iam[ iam.year == 2010 ]
iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
iam = iam.drop( columns = ['value', 'baseline'] )

### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions = regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
regions = regions.replace({'United States':'United States of America',
                           'Bolivia': 'Bolivia (Plurinational State of)',
                           'Libyan Arab Jamahiriya': 'Libya',
                           'Venezuela':'Venezuela (Bolivarian Republic of)',
                           'Russia':'Russian Federation',
                           'Cote dIvoire': "Côte d'Ivoire",
                           'Laos': "Lao People's Democratic Republic",
                           'Burma':'Myanmar',
                           'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                           'Korea,  Republic of': 'Republic of Korea'})

### livestock numbers from FAO ###
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
#ruminants = ['Goat','Sheep','Cattle','Buffaloes']
ruminants = ['Cattle']
df = df[df.Item.isin(ruminants)]
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[ df.year==2010 ]
df = df.drop( columns = ('year') )
df = df.groupby( ['REGION'], as_index = False ).sum().rename(columns = {'value':'cattle2010'})
df.cattle2010 = df.cattle2010 / 1e6

### Combine country level livestock data with regional ssp projected changes ###
df = pd.merge(df, iam, on = ['REGION'])
df['cattle'] = df.cattle2010 * (1 + df.change / 100)
df['addcattle'] = (df.cattle - df.cattle2010)

df = df[df['SCENARIO'].str.contains('Baseline')]
df = df[df.year>2005]

# 2. Land Use Change at the country level --> merge with regional livestock projections


In [ ]:
### pick an ssp ###
ssp = 370     ###
### pick an ssp ###

### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
dfc = dfc[dfc.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )
smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
######----------------------------------------------------------------------------------------------#####

### Land use data from LUH2 ###
dfla = pd.read_csv( f'/content/drive/My Drive/data/livestock/land_use/ssp{ssp}_country_level.csv' ) ###ssp370
dfla = pd.merge( dfla, dfc, on = 'country' ).drop(columns = ['countryname',])
dfla = pd.merge(dfla, regions, on = 'ISO').rename( columns = {'R5_region':'REGION'})
dfla['land'] = dfla.pastr + dfla.rang + dfla.crop

dflabase = dfla[dfla.year == 2015]
dflabase = dflabase[['ISO','land']].rename(columns ={ 'land': 'land2015'})
dfla = pd.merge( dfla, dflabase, on = 'ISO' )
dfla['landchange'] = dfla.land - dfla.land2015 

dflaO= dfla[dfla.REGION == 'R5.2OECD']
dflaregO = dflaO.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
dflaO = pd.merge( dflaO, dflaregO, on = ['REGION', 'year'] )
dflaO['contribution'] = dflaO.landchange / dflaO.region_landchange

dfla = dfla[dfla.REGION != 'R5.2OECD']
dflareg = dfla[ dfla.landchange > 0 ] ### only take positive land change areas ###
dflareg = dflareg.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
dfla = pd.merge( dfla, dflareg, on = ['REGION', 'year'] )
dfla['contribution'] = dfla.landchange / dfla.region_landchange

dfla = pd.concat( (dfla, dflaO) )

decades = [2015, 2025, 2035, 2045, 2055, 2065, 2075, 2085, 2095]
dfnew = pd.DataFrame({})
for year in decades:
    dft = dfla[dfla.year>year]
    dft = dft[dft.year<year+10]
    dft = dft.groupby( ['ISO','Country','REGION'], as_index = False ).mean()
    dfnew = pd.concat( (dfnew, dft) )

dfnew = dfnew.replace(2097.5, 2100)
dfnew.year = dfnew.year.astype(int)
dfnew.head(3)

### combine regional animal gains with country-level land change
df = df[ df.SCENARIO == 'SSP1-Baseline' ] ######## CHANNNNNNGEEEEEE MEEEEEEEEEEE #####
df = pd.merge( dfnew, df, on = ['REGION','year'] )
df['newcows'] = df.addcattle * df.contribution

### livestock numbers from FAO to get country level in 2010 again ###
dfa = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfa = dfa.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
dfa = dfa.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
dfa.year = dfa.year.apply( lambda x: x.split('Y')[1] ).astype( int )
dfa = dfa[dfa.Element == 'Stocks']
dfa = dfa[dfa.Item.isin(['Cattle'])]
dfa = dfa[dfa.Country != 'World']
dfa = dfa[ dfa.year == 2010 ]
dfa = pd.merge( dfa, regions, on = ['Country'] )
dfa = dfa[['ISO', 'value']].rename( columns = {'value':'cows_country_2010'} )
dfa.cows_country_2010 = dfa.cows_country_2010 / 1e6

df = pd.merge(df, dfa, on = ['ISO'])

### remove negative projections ###
#df.newcows[df.newcows < 0 ] = 0
dfO = df[df.REGION == 'R5.2OECD']
df = df[df.REGION != 'R5.2OECD']
df.newcows[df.newcows < 0 ] = 0
df = pd.concat(( df, dfO ))
df['projected_cows'] = (df.newcows + df.cows_country_2010)

### ditch Reforming economies ###
df = df[df.REGION != 'R5.2REF']
df.head(10)

# 3. Where to put the cows?

In [ ]:
cows = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
cows[ cows < 1 ] = 0
cows = xr.DataArray( cows, dims = {'lat': np.linspace( 90, -90, cows.shape[0] ), 'lon':  np.linspace( -180, 180, cows.shape[1] ) } )
cows = cows.coarsen( lat = 3, lon = 3, boundary = 'trim' ).sum().compute()
cows2015 = cows.values

cellarea = np.repeat(cellarea[np.newaxis,:,:], 86, axis = 0)

mask = xr.load_dataarray('/content/drive/Shared drives/ClimateAg/data/gadm36_level0_0_25_180_180.nc4')
mask = np.flip( np.nan_to_num( mask.values ), axis = 0 )

ssp370 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-AIM-ssp370-2-1-f_gn_2015-2100.nc'
ssp126 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp126-2-1-f_gn_2015-2100.nc'
land   = xr.open_dataset( ssp370, decode_times = False )
past = np.nan_to_num( land.pastr.values ) * cellarea 
rang = np.nan_to_num( land.range.values ) * cellarea
c3   = np.nan_to_num( land.c3ann.values ) * cellarea
c4   = np.nan_to_num( land.c4ann.values ) * cellarea
c3n  = np.nan_to_num( land.c3nfx.values ) * cellarea
land = past + rang + c3 + c4 + c3n

### Calculate decadal means ###
ids = [0, 10, 20, 30, 40, 50, 60, 70, 80]
landd = np.zeros( ( 9, 720, 1440 ) )
for i, year in enumerate( ids ): landd[i, :, :] = land[ year:year+9, :, : ].mean( axis = 0 )
land2015 = land[0,:,:]
land = landd - land2015

In [ ]:
years = [2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]
cows370 = np.ma.zeros( ( 10, 720, 1440 ) )
cows370[0,:,:] = cows2015.copy()

for i in range(9):
    cows_temp = cows2015.copy()
    df1 = df[ df.year == years[i] ]
    countries_with_cows = df1.country.unique()
    for j in countries_with_cows:
        add_cows = df1[df1.country == j].newcows.values[0] * 1e6
        if ((add_cows < -1) | (add_cows > 1)):
            newcows = np.nan_to_num( land[i,:,:].copy() )
            newcows[ mask != j ] = 0
            total_area = np.sum( newcows )
            newcows = add_cows * ( newcows / total_area ) 
            cows_temp += newcows
        else: pass

    cows370[i+1,:,:] = cows_temp

newmask = np.zeros_like( mask )
newmask[:,:720] = mask[:,720:]
newmask[:,720:] = mask[:,:720]
newmask = np.flip( newmask, axis = 0 )
countries = np.unique(newmask)
countries = countries[~np.isnan(countries)]

cows = np.zeros_like( cows370 )
cows[:,:,:720] = cows370[:,:,720:]
cows[:,:,720:] = cows370[:,:,:720]
cows = np.flip( cows, axis = 1 )

del land, landd, cows370

# Temperature Data Processing

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )

df1 = df.query( "activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day'" )
hist_models = list( set(tas).intersection( set(df1.source_id.unique().tolist()) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp126' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp126' & table_id == 'day'" )
ssp126_models = list( set(tas).intersection( set(df1.source_id.unique().tolist()) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp245' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp245' & table_id == 'day'" )
ssp245_models = list( set(tas).intersection( set(df1.source_id.unique().tolist()) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp370' & table_id == 'Amonday'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp370' & table_id == 'day'" )
ssp370_models = list( set(tas).intersection( set(df1.source_id.unique().tolist()) ) )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp585' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp585' & table_id == 'day'" )
ssp585_models = list( set(tas).intersection( set(df1.source_id.unique().tolist()) ) )


In [ ]:
def calc_ssp(models, ssp):
    years = [2085] #np.arange( 2015, 2100, 10 )
    pro_thresh = 68     # production threshold
    for model in models:
        dfout = []
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

        for i, year in enumerate( years ):
            print(model, year)
            hu  = huss.sel(time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).huss.values
            tas = Ta.sel(  time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).tas.values - 273.15
            RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
            RH = xr.where( RH < 1, RH, 1 )
            del hu
            thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
            del tas, RH

            p = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )      ### MEAN == severity ###
            p = cv2.resize( p, ( cows.shape[2], cows.shape[1] ) )

            for country in countries:
                prod = np.ma.masked_where( newmask!=country, p ).harden_mask()
                prod = np.ma.average( prod, weights = cows[-2,:,:])

                dfout.append( {'year' : year, 'country': country, 'produ': prod } )

        dfout = pd.DataFrame( dfout )
        dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_{ssp}_HL_dynamic_cows_68_2090.csv', index = False )
        print(dfout.head(3))

In [ ]:
cows = np.zeros_like( cows2015 )
cows[:,:720] = cows2015[:,720:]
cows[:,720:] = cows2015[:,:720]
cows2015f = np.flip( cows, axis = 0 )

In [ ]:
def calc_ssp(models, ssp):
    years = [2045] #np.arange( 2015, 2100, 10 )
    pro_thresh = 68     # production threshold
    for model in models:
        dfout = []
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

        for i, year in enumerate( years ):
            print(model, year)
            hu  = huss.sel(time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).huss.values
            tas = Ta.sel(  time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).tas.values - 273.15
            RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
            RH = xr.where( RH < 1, RH, 1 )
            del hu
            thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
            del tas, RH

            p = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )      ### MEAN == severity ###
            p = cv2.resize( p, ( cows2015f.shape[1], cows2015f.shape[0] ) )

            for country in countries:
                prod = np.ma.masked_where( newmask!=country, p ).harden_mask()
                #prod = np.ma.average( prod, weights = cows[-2,:,:])
                prod = np.ma.masked_where( cows2015f < 1, prod ).compressed().ravel()
                prod = np.sort(prod)
                size = np.floor(np.shape(prod)[0]/3).astype(int)
                prod = np.nanmean(prod[:size])

                dfout.append( {'year' : year, 'country': country, 'produ': prod } )

        dfout = pd.DataFrame( dfout )
        dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_{ssp}_HL_cold_cows_68_2050.csv', index = False )
        print(dfout.head(3))

In [ ]:
models = ['KACE-1-0-G', 'BCC-CSM2-MR', 'UKESM1-0-LL', 'CNRM-CM6-1', 'GFDL-ESM4', 'CanESM5',
                 'EC-Earth3-Veg', 'MRI-ESM2-0', 'CESM2-WACCM', 'MPI-ESM1-2-HR']

calc_ssp( models, 'ssp126' )

In [ ]:
def calc_ssp(models, ssp):
    years = [2045] #np.arange( 2015, 2100, 10 )
    pro_thresh = 68     # production threshold
    for model in models:
        dfout = []
        df1 = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )


        print(model, year)
        hu  = huss.sel(time = slice( f'2005-01-01T12:00:00', f'2014-12-30T12:00:00') ).huss.values
        tas = Ta.sel(  time = slice( f'2005-01-01T12:00:00', f'2014-12-30T12:00:00') ).tas.values - 273.15
        RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
        RH = xr.where( RH < 1, RH, 1 )
        del hu
        thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
        del tas, RH

        p = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )      ### MEAN == severity ###
        p = cv2.resize( p, ( cows2015f.shape[1], cows2015f.shape[0] ) )

        for country in countries:
            prod = np.ma.masked_where( newmask!=country, p ).harden_mask()
            #prod = np.ma.average( prod, weights = cows[-2,:,:])
            prod = np.ma.masked_where( cows2015f < 1, prod ).compressed().ravel()
            prod = np.sort(prod)
            size = np.floor(np.shape(prod)[0]/3).astype(int)
            prod = np.nanmean(prod[:size])

            dfout.append( {'year' : year, 'country': country, 'produ': prod } )

        dfout = pd.DataFrame( dfout )
        dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_hist_HL_cold_cows_68.csv', index = False )
        print(dfout.head(3))

In [ ]:
calc_ssp( models, 'hist' )

## ERA historical temp to country level



In [ ]:
cows = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
cows[cows < 1] = 0
newcows = np.zeros_like(cows)
newcows[:,:2160] = cows[:,2160:]
newcows[:,2160:] = cows[:,:2160]
cows = cv2.resize( newcows, ( 1440, 719 ) )

era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
era = era.values
era = np.sum( np.ma.masked_where( era < 68, era ) - 68, axis = 0 )
era = era[1:-1,:]
era = era.filled( 0 )

mask = xr.load_dataarray('/content/drive/My Drive/data/area_masks/gadm36_level0_8975_8975_0_35975.nc4')
mask = mask.values
countries = np.unique( mask )
countries = countries[ ~np.isnan(countries) ]

dfout = []
for country in countries:
    prod = np.ma.masked_where( mask!=country, era ).harden_mask()
    prod = np.ma.average( prod, weights = cows)
    dfout.append( {'country': country, 'produ': prod } )

dfout = pd.DataFrame( dfout )
dfout.to_csv(f'/content/drive/My Drive/data/livestock/era_2010_HL_static_cows_68.csv', index = False )


In [ ]:
models = ['KACE-1-0-G', 'BCC-CSM2-MR', 'UKESM1-0-LL', 'CNRM-CM6-1', 'GFDL-ESM4', 'CanESM5',
                 'EC-Earth3-Veg', 'MRI-ESM2-0', 'CESM2-WACCM', 'MPI-ESM1-2-HR']

calc_ssp( models, 'ssp370' )

In [ ]:
models = ['MRI-ESM2-0','EC-Earth3-Veg','UKESM1-0-LL',
 'BCC-CSM2-MR','GFDL-ESM4','CanESM5','MPI-ESM1-2-HR','CESM2-WACCM',
 'CNRM-CM6-1','KACE-1-0-G','CNRM-ESM2-1']

calc_ssp( models, 'ssp126' )

In [ ]:
models = ['KACE-1-0-G', 'BCC-CSM2-MR', 'UKESM1-0-LL', 'CNRM-CM6-1', 'GFDL-ESM4', 'CanESM5',
                 'EC-Earth3-Veg', 'MRI-ESM2-0', 'CESM2-WACCM', 'MPI-ESM1-2-HR']

years = [ 1985, 1995, 2005 ]
pro_thresh = 68 
for model in models:
    dfout = []
    df1 = df.query(  f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1 = df.query(  f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'"  )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    for year in years:
        print(model, year)
        hu  = huss.sel(time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).huss.values
        tas = Ta.sel(  time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).tas.values - 273.15
        RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
        RH = xr.where( RH < 1, RH, 1 )
        del hu
        thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
        del tas, RH

        thi = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )      
        thi = cv2.resize( thi, (cows.shape[2], cows.shape[1]) )

        for country in countries:
            prod = np.ma.masked_where( newmask != country, thi ).harden_mask()
            prod = np.ma.average( prod, weights = cows[0,:,:] )
            dfout.append( {'year' : year, 'country': country, 'produ': prod } )

    dfout = pd.DataFrame( dfout )
    dfout = dfout.groupby( ['country'], as_index = False ).mean()

    dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_historical_HL_68.csv', index = False )


# OLD


---



---



---



In [ ]:
### Process the LUH2 data to country level ###
ssp585 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc'
ssp370 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-AIM-ssp370-2-1-f_gn_2015-2100.nc'
ssp126 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp126-2-1-f_gn_2015-2100.nc'

mask = xr.load_dataarray('/content/drive/Shared drives/ClimateAg/data/gadm36_level0_0_25_180_180.nc4')
mask = np.flip( np.nan_to_num( mask.values ), axis = 0 )

x = xr.open_dataset(ssp126, decode_times=False)
pastr = np.nan_to_num( x.pastr.values ) * cellarea 
rang  = np.nan_to_num( x.range.values ) * cellarea
c3  = np.nan_to_num( x.c3ann.values )   * cellarea
c4  = np.nan_to_num( x.c4ann.values )   * cellarea
c3n  = np.nan_to_num( x.c3nfx.values )  * cellarea
crop = c3 + c4 + c3n

years = np.arange( 2015, 2100, 1 )
df = []
for i in range( 85 ):
    print(i)
    for j in np.unique( mask ):
        df.append({'year':  years[i], 'country': j, 
                   'pastr': np.ma.masked_where(mask != j, pastr[i,:,:]).sum(),
                   'rang':  np.ma.masked_where(mask != j, rang[i,:,:]).sum(),
                   'crop':  np.ma.masked_where(mask != j, crop[i,:,:]).sum()})

df = pd.DataFrame( df )
df.to_csv( '/content/drive/My Drive/data/livestock/land_use/ssp126_country_level.csv', index = False )

In [ ]:
### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df[['Area','Item']]
df = df.rename( columns = {'Area':'countryname'} )
df = df[df.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'} )
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )

smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
df = df[ ~df.countryname.isin(smallisland) ]
dfc = pd.merge( df, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

In [ ]:
countrymaskgrid = xr.open_dataarray( '/content/drive/My Drive/data/area_masks/gadm36_countries_FOA_animal_dist_resolution.nc4' ).values

In [ ]:
def calc_ssp(models, ssp):
    years = np.arange( 2015, 2100, 10 )
    pro_thresh = 69     # production threshold
    #dea_thresh = 73     # death and fertility threshold
    for model in models:
        dfout = []
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
        df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
        Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

        if huss.lat.shape[0] == Ta.lat.shape[0]:
            for year in years:
                print(model, year)
                hu  = huss.sel(time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).huss.values
                tas = Ta.sel(  time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).tas.values - 273.15
                RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
                RH = xr.where( RH < 1, RH, 1 )
                del hu
                thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
                del tas, RH

                p = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )       
                p = cv2.resize( p, (cows.shape[1], cows.shape[0]) )

                for country in countries:
                    prod = np.ma.masked_where( newmask!=country, p ).harden_mask()
                    prod = np.ma.average(prod, weights = newcow )

                    dfout.append( {'year' : year, 'country': country, 'produ': prod } )

            dfout = pd.DataFrame(dfout)

            dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_{ssp}.csv', index = False )

        else: print('bad', model)

In [ ]:
calc_ssp(ssp126_models, 'ssp126')
calc_ssp(ssp245_models, 'ssp245')
calc_ssp(ssp370_models, 'ssp370')
calc_ssp(ssp585_models, 'ssp585')

In [ ]:
ssp126_models

In [ ]:
years = np.arange( 1985, 2100, 10 )

In [ ]:
countrymaskgrid  = xr.open_dataarray( '/content/drive/My Drive/data/area_masks/gadm36_countries_FOA_animal_dist_resolution.nc4' ).values
newmask = np.zeros_like(countrymaskgrid)
newmask[:,:2160] = countrymaskgrid[:,2160:]
newmask[:,2160:] = countrymaskgrid[:,:2160]
countries = np.unique(newmask)
countries = countries[~np.isnan(countries)]
### Cattle Density data from FAO ###
cows = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
area = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/8_Areakm.tif', cv2.IMREAD_UNCHANGED)
cows = cows / area
cows = np.flip( cows, axis = 0 )
newcow = np.zeros_like(cows)
newcow[:,:2160] = cows[:,2160:]
newcow[:,2160:] = cows[:,:2160]
newcow[newcow<0] = 0

In [ ]:
models = ['EC-Earth3-Veg', 'CNRM-CM6-1-HR']

years = [ 1985, 1995, 2005 ]
pro_thresh = 69  
for model in models:
    dfout = []
    df1 = df.query(  f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1 = df.query(  f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'"  )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    for year in years:
        print(model, year)
        hu  = huss.sel(time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).huss.values
        tas = Ta.sel(  time = slice( f'{year}-01-01T12:00:00', f'{year+9}-12-30T12:00:00') ).tas.values - 273.15
        RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
        RH = xr.where( RH < 1, RH, 1 )
        del hu
        thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )
        del tas, RH

        thi = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )      
        thi = cv2.resize( thi, (cows.shape[1], cows.shape[0]) )

        for country in countries:
            prod = np.ma.masked_where( newmask != country, thi ).harden_mask()
            prod = np.ma.average( prod, weights = newcow )
            dfout.append( {'year' : year, 'country': country, 'produ': prod } )

    dfout = pd.DataFrame(dfout)
    dfout = dfout.groupby(['country'], as_index = False ).mean()

    dfout.to_csv(f'/content/drive/My Drive/data/livestock/cmip6_country/{model}_historical.csv', index = False )


## Zonal Means

In [ ]:
pro_thresh = 69 

ssp585_models = ['CESM2-WACCM',
 'KACE-1-0-G',
 'GFDL-CM4',
 'UKESM1-0-LL',
 'GFDL-ESM4',
 'EC-Earth3-Veg',
 'CNRM-ESM2-1',
 'BCC-CSM2-MR',
 'CanESM5',
 'CNRM-CM6-1-HR',
 'MPI-ESM1-2-HR',
 'CNRM-CM6-1',
 'MRI-ESM2-0']

ssp = 'ssp585'

for model in ssp585_models:
    print(model)
    df1  = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1  = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == '{ssp}' & table_id == 'day' & source_id == '{model}'" )
    Ta   = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    hu  = huss.sel( time = slice( '2090-01-01T12:00:00', '2099-12-30T12:00:00') ).huss.values
    tas = Ta.sel(   time = slice( '2090-01-01T12:00:00', '2099-12-30T12:00:00') ).tas.values - 273.15

    RH  = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
    del hu
    RH  = xr.where(RH < 1, RH, 1)
    thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )

    pf = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )
    tasf = np.mean( tas, axis = 0 )
    del tas, RH, thi

    df1  = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1  = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta   = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    hu  = huss.sel(time = slice( '2005-01-01T12:00:00', '2014-12-30T12:00:00') ).huss.values
    tas = Ta.sel(  time = slice( '2005-01-01T12:00:00', '2014-12-30T12:00:00') ).tas.values - 273.15

    RH  = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
    del hu
    RH  = xr.where(RH < 1, RH, 1)
    thi = (1.8 * tas + 32) - ( (0.55 - 0.55 * RH) * (1.8 * tas - 26) )

    ph = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )
    tash = np.mean( tas, axis = 0 )
    del tas, RH, thi

    tas = tasf - tash
    p   = pf / 3650 - ph / 3650

    lon = Ta.lon.values
    lat = Ta.lat.values
    lon2d, lat2d = np.meshgrid( lon, lat )

    tas = maskoceans( lon2d, lat2d, tas )
    p   = maskoceans( lon2d, lat2d, p )

    tas = tas.mean(axis = 1)
    p   = p.mean(axis = 1)

    out = np.zeros( ( lat.shape[0], 3 ) )
    out[:,0] = lat
    out[:,1] = tas
    out[:,2] = p

    np.savetxt( f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean.csv', out, delimiter=',' )

# OLD


---



---



---



In [ ]:
### filter out to just get some lons for africa ###
lon1 = np.argmin(np.abs(lon + 19))
lon2 = np.argmin(np.abs(lon - 52))
tas[:,:lon1] = -999
tas[:,lon2:] = -999
p[:,:lon1] = -999
p[:,lon2:] = -999
tas = np.ma.masked_where(tas == -999, tas)
p = np.ma.masked_where(tas == -999, p) 
### filter out to just get some lons for africa ###

In [ ]:
years = np.arange( 2015, 2100, 10 )
pro_thresh = 67.9     # production threshold
dea_thresh = 73       # death and fertility threshold

for model in ssp126_models:
    df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp126' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp126' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    hu  = huss.sel(time = slice( '2090-01-01T12:00:00', '2100-12-30T12:00:00') ).huss.values
    tas = Ta.sel(  time = slice( '2090-01-01T12:00:00', '2100-12-30T12:00:00') ).tas.values - 273.15
    RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
    RH = xr.where( RH < 1, RH, 1 )
    thi = 0.80 * tas + tas * RH - 1.4 * RH + 46.4
    del hu, RH

    pf = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )
    tasf = np.mean( tas, axis = 0 )

    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'huss' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    huss = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    df1 = df.query( f"activity_id=='CMIP' & variable_id == 'tas' & experiment_id == 'historical' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )

    hu  = huss.sel(time = slice( '2005-01-01T12:00:00', '2014-12-30T12:00:00') ).huss.values
    tas = Ta.sel(  time = slice( '2005-01-01T12:00:00', '2014-12-30T12:00:00') ).tas.values - 273.15
    RH = (hu * 1013 / (0.378 * hu + 0.622)) / (6.112 * np.exp((17.67 * tas)/(tas + 243.5)))
    RH = xr.where( RH < 1, RH, 1 )
    thi = 0.80 * tas + tas * RH - 1.4 * RH + 46.4
    del hu, RH

    ph = np.sum( np.ma.masked_where( thi < pro_thresh, thi ) - pro_thresh, axis = 0 )
    tash = np.mean( tas, axis = 0 )

    tas = tasf - tash
    p = pf / 3650 - ph / (3650)

    lon = Ta.lon.values
    lat = Ta.lat.values
    lon2d, lat2d = np.meshgrid( lon, lat )

    tas = maskoceans( lon2d, lat2d, tas )
    p = maskoceans( lon2d, lat2d, p )

    ### filter out to just get some lons for africa ###
    lon1 = np.argmin(np.abs(lon + 19))
    lon2 = np.argmin(np.abs(lon - 52))
    tas[:,:lon1] = -999
    tas[:,lon2:] = -999
    p[:,:lon1] = -999
    p[:,lon2:] = -999
    tas = np.ma.masked_where(tas == -999, tas)
    p = np.ma.masked_where(tas == -999, p) 
    ### filter out to just get some lons for africa ###

    tas = tas.mean(axis = 1)
    p = p.mean(axis = 1)

    out = np.zeros( ( lat.shape[0], 3 ) )
    out[:,0] = lat
    out[:,1] = tas
    out[:,2] = p
    print(model)
    np.savetxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean_africa.csv', out, delimiter=',')

In [ ]:
models = ssp585_models
newlat = np.linspace(-90,90,400)
tas = np.zeros( (400, len(models)) )
thi = np.zeros( (400, len(models)) )

red = '#980000'

for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=2)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    tas[:,i] = spl1(newlat)
    thi[:,i] = spl2(newlat)

tasmin = np.min(tas, axis = 1)
tasmax = np.max(tas, axis = 1)
tasmed = np.mean(tas, axis = 1)

thimin = np.min(thi, axis = 1)
thimax = np.max(thi, axis = 1)
thimed = np.mean(thi, axis = 1)

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (1, 2 ) )

newlat = np.linspace(-90,90,400)

color = '#bf9000' # '#4a86e8'

for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = color, lw = 0.4, alpha = 0.8, zorder = 1)

plt.fill_betweenx(newlat, tasmin, tasmax, facecolor = 'black', alpha = 0.15, zorder = 2)
plt.fill_betweenx(newlat, thimin, thimax, facecolor = color, alpha = 0.2, zorder = 3)

plt.plot(tasmed, newlat, color = 'black', lw = 1.5, alpha = 1, zorder = 4)
plt.plot(thimed, newlat, color = color, lw = 2, alpha = 1, zorder = 5)

plt.xlim(0,15)
plt.xticks([0,5,10,15],[0,5,10,15])
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.ylim(-56,76)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/ssp585_zonal_mean.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
models = ssp126_models
newlat = np.linspace(-90,90,400)
tas = np.zeros( (400, len(models)) )
thi = np.zeros( (400, len(models)) )

red = '#980000'

for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=2)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    tas[:,i] = spl1(newlat)
    thi[:,i] = spl2(newlat)

tasmin = np.min(tas, axis = 1)
tasmax = np.max(tas, axis = 1)
tasmed = np.mean(tas, axis = 1)

thimin = np.min(thi, axis = 1)
thimax = np.max(thi, axis = 1)
thimed = np.mean(thi, axis = 1)

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (1, 2 ) )

newlat = np.linspace(-90,90,400)

color = '#bf9000' # '#4a86e8'

for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = color, lw = 0.4, alpha = 0.8, zorder = 1)

plt.fill_betweenx(newlat, tasmin, tasmax, facecolor = 'black', alpha = 0.15, zorder = 2)
plt.fill_betweenx(newlat, thimin, thimax, facecolor = color, alpha = 0.2, zorder = 3)

plt.plot(tasmed, newlat, color = 'black', lw = 1.5, alpha = 1, zorder = 4)
plt.plot(thimed, newlat, color = color, lw = 2, alpha = 1, zorder = 5)

plt.xlim(0,15)
plt.xticks([0,5,10,15],[0,5,10,15])
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.ylim(-56,76)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/ssp126_zonal_mean.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
models = ssp585_models
newlat = np.linspace(-90,90,400)
thi = np.zeros( (400, len(models)) )

red = '#980000'

for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean_africa.csv', delimiter=',')
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    thi[:,i] = spl2(newlat)

thimin = np.min(thi, axis = 1)
thimax = np.max(thi, axis = 1)
thimed = np.mean(thi, axis = 1)

models = ssp126_models
thi1 = np.zeros( (400, len(models)) )
for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean_africa.csv', delimiter=',')
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    thi1[:,i] = spl2(newlat)

thi1min = np.min(thi1, axis = 1)
thi1max = np.max(thi1, axis = 1)
thi1med = np.mean(thi1, axis = 1)

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (2.75, 4 ) )

newlat = np.linspace(-90,90,400)

color = '#bf9000' # '#4a86e8'

for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean_africa.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    #plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 1)

models = ssp126_models
for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean_africa.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    #plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = color, lw = 0.4, alpha = 0.8, zorder = 1)

plt.fill_betweenx(newlat, thi1min, thi1max, facecolor = 'black', alpha = 0.15, zorder = 2)
plt.fill_betweenx(newlat, thimin, thimax, facecolor = color, alpha = 0.2, zorder = 3)

plt.plot(thi1med, newlat, color = 'black', lw = 1.5, alpha = 1, zorder = 4)
plt.plot(thimed, newlat, color = color, lw = 2, alpha = 1, zorder = 5)

plt.xlim(0,12.5)
plt.xticks([0,2.5,5,7.5,10,12.5],[0,2.5,5,7.5,10,12.5])
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.ylim(-34,37.5)
sns.despine( offset = 5, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/ssp585_zonal_mean.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )